This is the same pipeline as in `07` and `08`, just putting it into a different database schema (`schema2.sql`) and running it on a computer where memory does not matter that much. :-)

In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
import sqlalchemy
from sqlalchemy import create_engine, text
from datetime import date

import os
from enum import Enum
from glob import glob
import logging
from typing import List
from itertools import combinations, chain, compress
import time

In [2]:
logging.basicConfig(filename="09-pipeline-schema2.log", level=logging.INFO,
                    format="%(asctime)s %(levelname)-8s %(message)s")

In [3]:
engine_string = "mysql://bukanuser@localhost/bukan?charset=utf8mb4"

In [4]:
def run_sql_query(query: str):
    engine = create_engine(engine_string, convert_unicode=True)
    with engine.connect() as conn:
        results = conn.execute(text(query)).fetchall()
    engine.dispose()
    return results

In [5]:
def log_progress(sequence, every=None, size=None, name='Items'):
    """From <https://github.com/kuk/log-progress>"""
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [6]:
overview = pd.read_csv("bukan-overview-extended.csv", index_col=0)

In [7]:
overview

,公開時期,オープンデータ分類,書名（統一書名）,刊・写,原本請求記号,刊年・書写年,（西暦）,冊数等,(単位),Pages per Scan,Aspect,TitleHiragana,TitleRomanji,NrImages,Estimate
国文研書誌ID,,,,,,,,,,,,,,,
200018462,H29.12,政治・法制,鎌倉武鑑,刊,ＭＹ－１４９０－４,文政２,1819,1.0,冊,2,Portrait,かまくらぶかん,kamakurabukan,69,False
200018466,H29.12,政治・法制,応仁武鑑,刊,ＭＹ－１４９０－７,天保１５,1844,2.0,冊,2,Portrait,おうにんぶかん,ōninbukan,78,False
200018476,H29.12,政治・法制,応仁武鑑,刊,ＭＹ－１４９０－８,弘化３,1846,3.0,冊,2,Portrait,おうにんぶかん,ōninbukan,100,False
200018713,H29.12,政治・法制,本朝武鑑,刊,ＭＹ－１２０１－５３,貞享３,1686,1.0,冊,2,Landscape,ほんちょうぶかん,honchōbukan,75,True
200018714,H29.12,政治・法制,本朝武鑑,刊,ＭＹ－１２０１－５４,貞享３,1686,1.0,冊,2,Landscape,ほんちょうぶかん,honchōbukan,94,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200019654,H29.12,政治・法制,懐宝御国分略武鑑,刊,ＭＹ－１２０１－３１８,慶応４,1868,1.0,冊,2,Landscape,かいほうおくにわけりゃくぶかん,kaihō okuniwake ryakubukan,24,False
200019661,H29.12,政治・法制,昇栄武鑑,刊,ＭＹ－１２０１－３２４,嘉永６,1853,1.0,冊,2,Landscape,しょうえいぶかん,shōeibukan,124,False
200019662,H29.12,政治・法制,昇栄武鑑,刊,ＭＹ－１２０１－３２５,安政３,1856,1.0,冊,2,Landscape,しょうえいぶかん,shōeibukan,125,False


## Putting metadata into database

In [8]:
title = overview.groupby(["書名（統一書名）", "TitleHiragana", "TitleRomanji"]).mean()
title = title.index.to_frame(index=False)
title.columns = ["kanji", "hiragana", "romanji"]
title.index = pd.RangeIndex(1,45)
title.index.name = "id"

In [9]:
engine = create_engine(engine_string)
#title.to_sql("title", engine, if_exists="append")
title = pd.read_sql_table("title", engine, index_col="id")
engine.dispose()

In [10]:
title_kanji_to_id = {kanji:id for id, kanji in title["kanji"].items()}

In [11]:
released = overview["公開時期"].apply(lambda x: date(2017, 12, 1) if x == "H29.12" else date(2015, 11, 1))
title_id = overview["書名（統一書名）"].apply(lambda x: title_kanji_to_id[x])
original_id = overview["原本請求記号"]
era_name = overview["刊年・書写年"].apply(lambda x: x[:2])
era_year = overview["刊年・書写年"].apply(lambda x: int(x[2:]))
year = overview["（西暦）"]
estimate = overview["Estimate"]
nr_books = overview["冊数等"]
pages_per_scan = overview["Pages per Scan"]
aspect = overview["Aspect"].apply(lambda x: x[0])
nr_scans = overview["NrImages"]
book = pd.concat([released, title_id, original_id, era_name, era_year, year, estimate, nr_books, pages_per_scan, aspect, nr_scans], axis=1)
book.columns = ["released", "title_id", "original_id", "era_name", "era_year", "year", "estimate", "nr_books", "pages_per_scan", "aspect", "nr_scans"]
book.index.name = "id"

In [12]:
engine = create_engine(engine_string)
#book.to_sql("book", engine, if_exists="append")
book = pd.read_sql_table("book", engine, index_col="id")
engine.dispose()

In [13]:
engine = create_engine(engine_string)
index = 1
for book_id, book_metadata in overview.iterrows():
    break
    pages_per_scan = book_metadata["Pages per Scan"]
    image_paths = glob(f"data/{book_id}/image/*.jpg")
    image_filenames = [os.path.splitext(os.path.split(path)[1]) for path in image_paths]
    if pages_per_scan == 1:
        image_filenames = [name + "-w" + ext for name, ext in  image_filenames]
        image_filenames.sort()
    else:
        image_filenames = [name + "-l" + ext for name, ext in  image_filenames] + [name + "-r" + ext for name, ext in  image_filenames]
        image_filenames.sort()
    page = pd.DataFrame(image_filenames, columns=["filename"]).assign(book_id=book_id)
    page["lr"] = page["filename"].apply(lambda x: x[16])
    page["page"] = page["filename"].apply(lambda x: int(x[10:15]))
    page.index = pd.RangeIndex(index, index + len(page), name="id")
    index = index + len(page)
    page.to_sql("page", engine, if_exists="append")
page = pd.read_sql_table("page", engine, index_col="id")
engine.dispose()

## Processing Images

What I need to do now per image is:

1. Read, greyscale and crop all images
2. Split right/left page if necessary
3. Extract features

In [204]:
class Page(Enum):
    """Japanese reading order is from right to left."""
    whole =  b"w"
    left =   b"l"
    right  = b"r"

In [15]:
def crop_image(img):
    target_height = 660
    target_width = 990
    height, width = img.shape
    x1 = (width - target_width) // 2
    y1 = (height - target_height) // 2
    x2 = x1 + target_width
    y2 = y1 + target_height
    return img[y1:y2, x1:x2]

In [16]:
def read_image(path):
    img = cv.imread(path, flags=cv.IMREAD_REDUCED_GRAYSCALE_4)
    img = crop_image(img)
    return img

In [17]:
def split_image(img):
    height, width = img.shape
    assert width == 990
    half_width = width // 2
    return img[:, :half_width], img[:, half_width:]

In [18]:
def features_to_dataframe(keypoints: List[cv.KeyPoint], descriptors: np.ndarray, page_id: int):
    df = pd.DataFrame([(kp.pt[0], kp.pt[1], kp.size, kp.angle, kp.response, kp.octave, kp.class_id) for kp in keypoints],
                      columns=["x", "y", "size", "angle", "response", "octave", "class_id"])
    df = (df
          .assign(descriptor=[bytes(descriptors[i,:]) for i in range(descriptors.shape[0])])
          .assign(feature_nr=df.index)
          .assign(page_id=len(df)*[page_id]))
    return df

In [19]:
detector = cv.AKAZE_create(cv.AKAZE_DESCRIPTOR_MLDB_UPRIGHT, descriptor_size=0, threshold=0.005)
index = 1
all_features = []
for page_id, page_metadata in log_progress(page.iterrows(), every=100, size=len(page), name="Page"):
    break
    image_path = "data/{}/image/{}".format(page_metadata["book_id"], page_metadata["filename"])
    image_path = image_path[:-6] + image_path[-4:]
    image = read_image(image_path)
    if page_metadata["lr"] == b"l":
        image, _ = split_image(image)
    elif page_metadata["lr"] == b"r":
        _, image = split_image(image)
    keypoints, descriptors = detector.detectAndCompute(image, None)
    if len(keypoints) > 0:
        feature_df = features_to_dataframe(keypoints, descriptors, page_id)
        feature_df.index = pd.RangeIndex(index, index + len(feature_df), name="id")
        feature_df.to_sql("feature", engine, if_exists="append")
        index = index + len(feature_df)
        all_features.append(feature_df)

In [20]:
#feature = pd.concat(all_features)
#feature = pd.read_parquet("feature.parquet", engine="pyarrow")

## Processing Feature Pairs

First, I need to get all all book combinations as well as a fixed page offset. For each combination I need to run the full pipeline:

1. Find matching features
2. Filter features by their position
3. Compute the homography
4. Select features using the homography mask
4. **Don't threshold the features**
5. Save them to disk

In [192]:
book_radius = 4
book_pairs_lr = set()
book_pairs_w = set()
for i in range(book.index.size):
    lower_bound = i - book_radius
    lower_bound = 0 if lower_bound < 0 else lower_bound
    upper_bound = i + book_radius + 1
    upper_bound = book.index.size if upper_bound >= book.index.size else upper_bound
    for j in range(lower_bound, upper_bound):
        if j != i:
            book1 = book.iloc[i]
            book2 = book.iloc[j]
            if book1["pages_per_scan"] == book2["pages_per_scan"]:
                if book1["pages_per_scan"] == 2: 
                    book_pairs_lr.add(frozenset([book1.name, book2.name]))
                else:  # This should be 1
                    assert book1["pages_per_scan"] == 1
                    book_pairs_w.add(frozenset([book1.name, book2.name]))
book_pairs_lr = [sorted([book1, book2]) for book1, book2 in book_pairs_lr]
book_pairs_lr.sort()
book_pairs_w = [sorted([book1, book2]) for book1, book2 in book_pairs_w]
book_pairs_w.sort()

In [193]:
engine = create_engine(engine_string, convert_unicode=True)

In [206]:
def process_pages(left, right, engine):
    page_radius = 8
    rmax_page = right[-1][1]
    right_inv_dict = {rpage:rid for rid, rpage in right}
    page_id_pairs = []
    for lid, lpage in left:
        lower_bound = lpage - page_radius
        lower_bound = 1 if lower_bound < 1 else lower_bound
        upper_bound = lpage + page_radius
        upper_bound = rmax_page if upper_bound > rmax_page else upper_bound
        for rpage in range(lower_bound, upper_bound + 1):
            page_id_pairs.append((lid, right_inv_dict[rpage]))
    return page_id_pairs

In [207]:
def process_book_pairs(pairs, lr: Page, engine):
    statement = text("SELECT id, page FROM page WHERE book_id = :book AND lr = :lr")
    all_pairs = []
    for book1, book2 in pairs:
        with engine.connect() as conn:
            pages1 = conn.execute(statement, {"book": book1, "lr": lr.value}).fetchall()
            pages2 = conn.execute(statement, {"book": book2, "lr": lr.value}).fetchall()
            page_id_pairs = process_pages(pages1, pages2, engine)
            all_pairs.extend(page_id_pairs)
    return all_pairs

In [219]:
page_id_pairs = process_book_pairs(book_pairs_lr, Page.left, engine)
page_id_pairs.extend(process_book_pairs(book_pairs_lr, Page.right, engine))
page_id_pairs.extend(process_book_pairs(book_pairs_w, Page.whole, engine))

In [323]:
wradius = 100.
hradius = 100.
def is_near(points_pair):
    (x1, y1), (x2, y2) = points_pair
    return ((x1 - wradius) <= x2 <= (x1 + wradius)) and ((y1 - hradius) <= y2 <= (y1 + hradius))

In [259]:
def create_desc_array(feature_tuple):
    result = np.empty((len(feature_tuple), 61), dtype=np.uint8)
    for i, val in enumerate(feature_tuple):
        result[i,:] = np.frombuffer(val[-1], dtype=np.uint8)
    return result

In [437]:
matcher = cv.BFMatcher_create(normType=cv.NORM_HAMMING)
max_match_distance = 100
statement = text("SELECT id, x, y, descriptor FROM feature WHERE page_id = :page ORDER BY feature_nr")
dmatch_id = 1
with engine.connect() as conn:
    for ppid, (page1, page2) in log_progress(enumerate(page_id_pairs), every=1000, size=len(page_id_pairs), name="Pair"):
        break
        features1 = conn.execute(statement, {"page": page1}).fetchall()
        features2 = conn.execute(statement, {"page": page2}).fetchall()
        desc1 = create_desc_array(features1)
        desc2 = create_desc_array(features2)
        # 1. find_matches
        matches = matcher.radiusMatch(desc1, desc2, max_match_distance)
        matches = list(chain.from_iterable(matches))
        if len(matches) == 0:
            pagepair = (ppid+1, page1, page2, 0)
            conn.execute(text("INSERT INTO pagepair (id, first_page, second_page, nr_matches) VALUES {}".format(str(pagepair))))
            continue
        # 2. select_keypoints
        points1 = np.empty((len(matches), 2), dtype=np.float32)
        points2 = np.empty((len(matches), 2), dtype=np.float32)
        for i, match in enumerate(matches):
            points1[i, :] = features1[match.queryIdx][1:3]
            points2[i, :] = features2[match.trainIdx][1:3]
        zipped_points = zip(points1, points2)
        points_mask = np.fromiter(map(is_near, zipped_points), dtype=np.bool, count=len(matches))
        points1 = points1[points_mask]
        points2 = points2[points_mask]
        assert points1.shape == points2.shape
        if points1.size == 0:
            pagepair = (ppid+1, page1, page2, 0)
            conn.execute(text("INSERT INTO pagepair (id, first_page, second_page, nr_matches) VALUES {}".format(str(pagepair))))
            continue
        # 3. compute_homography
        h, h_mask = cv.findHomography(points1, points2, cv.RANSAC)
        if not h_mask.sum() > 0:
            pagepair = (ppid+1, page1, page2, 0)
            conn.execute(text("INSERT INTO pagepair (id, first_page, second_page, nr_matches) VALUES {}".format(str(pagepair))))
            continue
        # 4. filter_bad_homographies
        is_bad_h = np.any(np.isnan(h)) or np.any(np.absolute(h[2,:2]) > 0.001)
        if is_bad_h:
            if np.any(np.isinf(h)):
                pagepair = (ppid+1, page1, page2, 0)
                conn.execute(text("INSERT INTO pagepair (id, first_page, second_page, nr_matches) VALUES {}".format(str(pagepair))))
            else:
                pagepair = (ppid+1, page1, page2, 0, h[0,0], h[0,1], h[0,2], h[1,0], h[1,1], h[1,2], h[2,0], h[2,1], h[2,2])
                conn.execute(text("INSERT INTO pagepair VALUES {}".format(str(pagepair))))
            continue
        # 5. chose_relevant_matches
        h_mask = np.squeeze(h_mask).astype(np.bool)
        j = 0
        for i in range(len(points_mask)):
            if points_mask[i]:
                if not h_mask[j]:
                    points_mask[i] = False
                j += 1
        assert points_mask.sum() == h_mask.sum()
        relevant_matches = list(compress(matches, points_mask))
        relevant_pairs = [(idx, features1[match.queryIdx][0], features2[match.trainIdx][0])
                          for idx, match in zip(range(dmatch_id, dmatch_id+len(relevant_matches)), relevant_matches)]
        dmatch_id = dmatch_id + len(relevant_matches)
        # 6. Insert in database
        pagepair = (ppid+1, page1, page2, len(relevant_pairs), h[0,0], h[0,1], h[0,2], h[1,0], h[1,1], h[1,2], h[2,0], h[2,1], h[2,2])
        conn.execute(text("INSERT INTO pagepair VALUES {}".format(str(pagepair))))
        conn.execute(text("INSERT INTO dmatch VALUES {}".format(", ".join(str(x) for x in relevant_pairs))))